In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
#from models.MLP import MLP_classifier
from sklearn.model_selection import train_test_split
from MLP import MLP_classifier

In [3]:
train_X = np.load('./data/fashion_mnist/train_x.npy')
train_y = np.load('./data/fashion_mnist/train_y.npy')
valid_X = np.load('./data/fashion_mnist/valid_x.npy')
valid_y = np.load('./data/fashion_mnist/valid_y.npy')

test_X = np.load('./data/fashion_mnist/test_x.npy')

In [4]:
proba_list = []

num_epochs = 20
for i in range(5):
    train_images, train_labels = train_X, train_y

    valid_images, valid_labels = valid_X, valid_y

    # Preprocess the dataset (normalize and convert to tensors)
    test_images = test_X
    train_images, valid_images = train_images / 255.0, valid_images / 255.0
    test_images = test_images / 255.0

    train_images = torch.tensor(train_images, dtype=torch.float32).reshape(-1, 28*28)
    train_labels = torch.tensor(train_labels, dtype=torch.long)
    valid_images = torch.tensor(valid_images, dtype=torch.float32).reshape(-1, 28*28)
    valid_labels = torch.tensor(valid_labels, dtype=torch.long)
    test_images = torch.Tensor(test_images).reshape(-1, 28*28)

    # Create DataLoader for training and testing sets
    train_dataset = TensorDataset(train_images, train_labels)
    valid_dataset = TensorDataset(valid_images, valid_labels)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

    # Create the model
    input_size = 28*28
    num_classes = 10
    model = MLP_classifier(input_size, num_classes)

    # Set up the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model using the train method
    model.train(train_loader, valid_loader, criterion, optimizer, num_epochs)
    proba_list.append(model.predict_proba(test_images))

Epoch [1/20], Loss: 1.6168
Accuracy: 36.80%
Epoch [2/20], Loss: 1.5578
Accuracy: 39.20%
Epoch [3/20], Loss: 1.7092
Accuracy: 40.40%
Epoch [4/20], Loss: 1.6796
Accuracy: 41.90%
Epoch [5/20], Loss: 1.5799
Accuracy: 46.00%
Epoch [6/20], Loss: 1.6483
Accuracy: 49.60%
Epoch [7/20], Loss: 1.6487
Accuracy: 50.30%
Epoch [8/20], Loss: 1.5861
Accuracy: 49.30%
Epoch [9/20], Loss: 1.4915
Accuracy: 55.50%
Epoch [10/20], Loss: 1.6152
Accuracy: 54.50%
Epoch [11/20], Loss: 1.4924
Accuracy: 55.30%
Epoch [12/20], Loss: 1.5868
Accuracy: 56.10%
Epoch [13/20], Loss: 1.5226
Accuracy: 56.20%
Epoch [14/20], Loss: 1.5237
Accuracy: 56.50%
Epoch [15/20], Loss: 1.5237
Accuracy: 58.20%
Epoch [16/20], Loss: 1.4926
Accuracy: 58.00%
Epoch [17/20], Loss: 1.5230
Accuracy: 55.90%
Epoch [18/20], Loss: 1.5861
Accuracy: 54.90%
Epoch [19/20], Loss: 1.5526
Accuracy: 58.00%
Epoch [20/20], Loss: 1.6175
Accuracy: 55.80%
Epoch [1/20], Loss: 1.6779
Accuracy: 36.90%
Epoch [2/20], Loss: 1.6489
Accuracy: 40.20%
Epoch [3/20], Loss: 1

In [5]:
results = torch.zeros_like(proba_list[0])
for proba in proba_list:
    results += proba

In [6]:
predicted = results.argmax(-1)

In [7]:
result = pd.DataFrame({"index":np.arange(len(predicted)), "label": predicted})
result.to_csv("result_ensemble.csv", index=None)